In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns

In [2]:
df = pd.read_csv('final_data.csv')
df = df.drop('Unnamed: 0', axis = 1) # remove extra column

In [3]:
response_var = df[['SalePrice']] # separate the y from the features
feats = df.drop('SalePrice', axis = 1) # features

In [4]:
np.random.seed(1) # for reprodusability of results
Xtrain, Xtest, ytrain, ytest = train_test_split(feats, response_var)

In [12]:
regr_1 = RandomForestRegressor()
regr_1.fit(Xtrain, ytrain)

<ipython-input-12-3498f94b6ef8>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr_1.fit(Xtrain, ytrain)


RandomForestRegressor(n_estimators=200)

In [56]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [40,50,60,70, 80,88],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

grid_search = GridSearchCV(estimator = regr_1, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [57]:
grid_search.fit(Xtrain, ytrain)
best_grid = grid_search.best_estimator_
best_grid

grid_search.best_params_
#grid_accuracy = evaluate(best_grid, Xtrain, ytrain)
#print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 15.4min finished
C:\Users\ykath\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 40,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [58]:
grid_search.best_score_

0.8543797985826975

In [32]:
y_pred2 = grid_search.predict(Xtest)
mse2 = mean_squared_error(ytest,y_pred2)
rmse2 = np.sqrt(mse2)
rmse2

0.18879214491357066

In [13]:
y_pred1 = regr_1.predict(Xtest)
#y_pred1

In [14]:
mse1 = mean_squared_error(ytest,y_pred1)

In [15]:
rmse1 = np.sqrt(mse1)

In [16]:
rmse1

0.1448257042665818

In [18]:
regr_1.score(Xtest, ytest)

0.8748536900082253

In [60]:
from sklearn.model_selection import GridSearchCV
grid_para_tree = [{
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int)
}]
regr_1.set_params(random_state=108)
grid_search_tree = GridSearchCV(regr_1, grid_para_tree, cv=5, n_jobs=-1)
%time grid_search_tree.fit(Xtrain, ytrain)

C:\Users\ykath\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Wall time: 2min 25s


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=200,
                                             random_state=108),
             n_jobs=-1,
             param_grid=[{'min_samples_leaf': range(1, 10),
                          'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30])}])

In [61]:
grid_search_tree.best_score_

0.8484413335101901

In [62]:
df2 = pd.read_csv('C:/Users/ykath/Downloads/final_train.csv')
df2 = df2.drop('Unnamed: 0', axis = 1) # remove extra column

In [65]:
df3 = pd.read_csv('C:/Users/ykath/Downloads/final_test.csv')
df3 = df3.drop('Unnamed: 0', axis = 1) # remove extra column

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.394449,9.360741,5,6,1961,1961,0.000000,3,3,3,...,0,0,0,0,1,0,0,0,1,0
1,4.406719,9.565775,6,6,1958,1958,4.691348,3,3,3,...,0,0,0,0,1,0,0,0,1,0
2,4.317488,9.534668,5,5,1997,1998,0.000000,3,3,4,...,0,0,0,0,1,0,0,0,1,0
3,4.369448,9.208238,6,6,1998,1998,3.044522,3,3,3,...,0,0,0,0,1,0,0,0,1,0
4,3.784190,8.518392,8,5,1992,1992,0.000000,4,3,4,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,3.091042,7.568896,4,7,1970,1970,0.000000,3,3,3,...,0,0,0,0,1,0,0,0,1,0
1454,3.091042,7.546974,4,5,1970,1970,0.000000,3,3,3,...,0,0,0,0,1,0,0,0,0,0
1455,5.081404,9.903538,5,7,1960,1996,0.000000,3,3,3,...,0,0,0,0,1,0,0,0,0,0
1456,4.143135,9.253591,5,5,1992,1992,0.000000,3,3,4,...,0,0,0,0,1,0,0,0,1,0


In [66]:
response_var2 = df2[['SalePrice']] # separate the y from the features
feats2 = df2.drop('SalePrice', axis = 1) # features

In [68]:
#response_var3 = df3[['SalePrice']] # separate the y from the features
feats3 = df3 # features

In [69]:
regr_2 = RandomForestRegressor()
regr_2.fit(feats2, response_var2)

<ipython-input-69-f941ecb5d627>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr_2.fit(feats2, response_var2)


RandomForestRegressor()

In [71]:
np.random.seed(1) # for reprodusability of results
Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(feats2, response_var2)

In [72]:
regr_4 = RandomForestRegressor()
regr_4.fit(Xtrain2, ytrain2)

<ipython-input-72-6096c7f7141d>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr_4.fit(Xtrain2, ytrain2)


RandomForestRegressor()

In [82]:
y_pred4 = regr_4.predict(Xtest2)
mse4 = mean_squared_error(ytest2,y_pred4)
rmse4 = np.sqrt(mse4)
print("Training performance of Random Forest on the new features:",regr_4.score(Xtrain2, ytrain2))
print("Testing performance of Random Forest on the new features:",regr_4.score(Xtest2, ytest2))

Training performance of Random Forest on the new features: 0.9824342420283937
Testing performance of Random Forest on the new features: 0.9044204212789222


In [76]:
y_pred4 #predicted values of test data 

array([12.07237933, 12.04112939, 11.73470372, 11.9911547 , 11.88574989,
       11.95114625, 12.29321791, 12.05533494, 12.36961401, 11.64121823,
       12.16958639, 12.6745845 , 11.63887604, 11.56649772, 11.83426075,
       11.84827007, 12.02661904, 11.85029962, 12.19114658, 11.9564824 ,
       12.94650354, 12.42464302, 12.10678202, 11.75842136, 11.88138861,
       12.00307886, 11.81865615, 12.14154923, 12.02382975, 11.89820379,
       11.67544215, 11.75440467, 11.62430261, 11.59249319, 11.87807451,
       11.86780912, 11.64131969, 11.95856888, 12.46559948, 11.85047675,
       12.41566254, 11.78657162, 11.69919497, 12.17300962, 12.54924221,
       12.1909106 , 11.63924364, 12.77554807, 12.26764348, 12.41532969,
       12.03555383, 12.05263139, 11.69029723, 12.35458325, 12.1667416 ,
       11.48850868, 11.70205547, 11.90941603, 11.55299015, 11.48517196,
       12.08431473, 11.83518502, 12.55953069, 12.73643749, 12.2107015 ,
       12.33271014, 12.17961241, 11.73752237, 11.82673926, 11.65

In [74]:
y_pred3 = regr_2.predict(feats3)
y_pred3

array([11.73991116, 11.94465757, 12.06876205, ..., 11.97711596,
       11.64079392, 12.35209282])

In [88]:
y_pred_final = regr_4.predict(feats3)
y_pred_final

array([11.75144595, 11.91676278, 12.09052744, ..., 11.97963232,
       11.62094851, 12.36874111])

In [89]:
Id_test = pd.read_csv('C:/Users/ykath/Downloads/test.csv')['Id']
predicted_price = pd.Series(np.exp(y_pred_final))
submission = pd.DataFrame({'Id': Id_test, 'SalePrice': predicted_price})
submission.to_csv('submission.csv', index=False)

In [85]:
Id_test

,Unnamed: 0,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,4.394449,9.360741,5,6,1961,1961,0.000000,3,3,...,0,0,0,0,1,0,0,0,1,0
1,1,4.406719,9.565775,6,6,1958,1958,4.691348,3,3,...,0,0,0,0,1,0,0,0,1,0
2,2,4.317488,9.534668,5,5,1997,1998,0.000000,3,3,...,0,0,0,0,1,0,0,0,1,0
3,3,4.369448,9.208238,6,6,1998,1998,3.044522,3,3,...,0,0,0,0,1,0,0,0,1,0
4,4,3.784190,8.518392,8,5,1992,1992,0.000000,4,3,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,3.091042,7.568896,4,7,1970,1970,0.000000,3,3,...,0,0,0,0,1,0,0,0,1,0
1454,1455,3.091042,7.546974,4,5,1970,1970,0.000000,3,3,...,0,0,0,0,1,0,0,0,0,0
1455,1456,5.081404,9.903538,5,7,1960,1996,0.000000,3,3,...,0,0,0,0,1,0,0,0,0,0
1456,1457,4.143135,9.253591,5,5,1992,1992,0.000000,3,3,...,0,0,0,0,1,0,0,0,1,0
